<a href="https://colab.research.google.com/github/azizamirsaidova/abstractive_text_summarization/blob/main/Fine_tune_simplet5_absractive_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge
!pip install tf-estimator-nightly==2.8.0.dev2021122109
!pip install folium==0.2.1
!pip install simplet5 -q

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
import nltk
from random import sample
import torch
torch.cuda.empty_cache()
import os
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from random import sample
from rouge import Rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79794 sha256=207366a8d9453d11c8b72c77ed1ace1fce2b1c36b32f0dafbb763441f2dd03cd
  Stored in directory: /root/.cache/pip/wheels/00/0c/07/d7792a5444d5bb074361ac27da53cee9d5cce59a07fe9da5dd
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.14.0
    Uninstalling folium-0.14.0:
      Successfully uninstalled folium-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.6 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
geemap 0.32.0 requires folium>=0.13.0, but you have folium 0.2.1 which is incompatible.
  Pr

In [ ]:
trainning_df = pd.read_csv('/content/gdrive/My Drive/cnn_dailymail/train.csv')

MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()
trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])


In [ ]:
for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('===========================================================================================================\n\n')

Original Text : 
By . Lucy Waterlow . PUBLISHED: . 10:00 EST, 22 November 2013 . | . UPDATED: . 08:20 EST, 25 November 2013 . Sexy lingerie may be at the top of many women's Christmas lists this year but you'll need an especially rich and generous partner to receive an item from a new capsule collection in Selfridges. The department store will exclusively stock lingerie by label Bordelle whose collection includes one girdle dress costing an eye-watering £4,000. The bondage-inspired black garment is made with satin elastic bands, customised French embroidered lace, 18-carat gold plated decorations and buckles and hand-sewn Swarovski crystals. Decadent: Bordelle's Roxelana girdle dress costs £4,000 and is embellished with Swarovski crystals, left, a model wears a similar style, right . British designer, Alex Popa, who founded Bordelle, told Vogue the lingerie is worth the high price tag because it's made-to-measure using expensive materials. She said: 'The brand has been going for five y

In [ ]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [ ]:
df.head()

,target_text,source_text
0,"Bishop John Folda, of North Dakota, is taking ...",summarize: By . Associated Press . PUBLISHED: ...
1,Criminal complaint: Cop used his role to help ...,summarize: (CNN) -- Ralph Mata was an internal...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",summarize: A drunk driver who killed a young w...
3,Nina dos Santos says Europe must be ready to a...,summarize: (CNN) -- With a breezy sweep of his...
4,Fleetwood top of League One after 2-0 win at S...,summarize: Fleetwood are the only team still t...


In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
MAX_EPOCHS = 10

torch.cuda.memory_summary(device=None, abbreviated=False)
torch.utils.checkpoint

model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
            eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE],
            source_max_token_len=MAX_LEN,
            target_max_token_len=SUMMARY_LEN,
            batch_size=4, max_epochs=MAX_EPOCHS, use_gpu=True)

model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

model.load_model("t5","./"+model_path, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

outputs/simplet5-epoch-9-train-loss-0.6103-val-loss-1.921


In [ ]:
no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [ ]:
for index in sample(list(np.arange(len(trainning_article_ls))),5):
    print('Original Text : ')
    print(trainning_article_ls[index])

    Original_article = trainning_article_ls[index]

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    Original_Summary = trainning_highlight_ls[index]

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))

    Predicted_Summary = trainning_article_ls[index]

    rouge = Rouge()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([Predicted_Summary], Original_Summary)

    print("-----------------")

    print(rouge.get_scores(Predicted_Summary, Original_Summary))

    print("-----------------")

    print(BLEUscore)

    print("-----------------")

    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors


Original Text : 
By . Hayley O'keeffe . PUBLISHED: . 05:23 EST, 31 October 2013 . | . UPDATED: . 12:53 EST, 31 October 2013 . When artist Alan Dodd was asked by friends to paint a mural on a blank 50-foot-long wall at their home he transformed it into an optical illusion. Friendly faces behind windows and lifelike archways leading to landscapes are painted onto the cottage, in Eye, Suffolk. And the images give an illusion of a make-believe world within. Nationally-renowned artist Alan Dodd stands with the unusual mural that he painted on the side of a friend's home in Eye, Suffolk . The largest painting in the Eye mural is a blacksmith's forge, which takes in the skyline features of the town's church and castle. Quirky characters and also animals feature on other parts of the house side. More... The picture you never thought you'd see... Anna Wintour eating a burger and fries! Artist gives Vogue editor and her fashion friends a calorie-laden makeover . Rare Italian 'San Gennaro' collec

Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors


["Alan Dodd spent 55 days up a ladder to complete the stunningly detailed work on the side of a friend's home in Eye, Suffolk. Friendly faces behind windows and lifelike archways leading to landscapes are painted onto the house side. Mr Dodd says the finished product is a homage to the history of botany and horticulture."]
-----------------
[{'rouge-1': {'r': 0.8780487804878049, 'p': 0.1094224924012158, 'f': 0.1945945926239591}, 'rouge-2': {'r': 0.44, 'p': 0.04230769230769231, 'f': 0.07719298085564794}, 'rouge-l': {'r': 0.8292682926829268, 'p': 0.1033434650455927, 'f': 0.1837837818131483}}]
-----------------
2.3931422985475966e-05
-----------------


Not Fine tuned Predicted Summary Text : 
artist spent 55 days up a ladder painting the 50-foot-long wall. he says it is an homage to the history of the market town in eye, suffolk - and its history is reflected in the work 'against direct sunlight'


Original Text : 
Jailed: Joshua Sadler, 21, was sentenced to 12 months after admitting dan